In [29]:
import os
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

# Define the data generators
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, validation_split=0.2)
Val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('/kaggle/input/shitpy/dataset/train', target_size=(64, 64), batch_size=32, class_mode='categorical', subset='training')
Val_generator = train_datagen.flow_from_directory('/kaggle/input/shitpy/dataset/train', target_size=(64, 64), batch_size=32, class_mode='categorical', subset='validation')

# Define the model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flatten layer to transition from convolutional to fully connected layers
model.add(Flatten())

# Fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))  # 5 classes, so the output layer has 5 neurons with softmax activation

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

# Train the model
model.fit(train_generator, epochs=15, steps_per_epoch=len(train_generator), validation_data=Val_generator, validation_steps=len(Val_generator))
model.save('classification_net.h5')

Found 7920 images belonging to 5 classes.
Found 1980 images belonging to 5 classes.
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 31, 31, 32)        0         
 ng2D)                                                           
                                                                 
 dropout_20 (Dropout)        (None, 31, 31, 32)        0         
                                                                 
 conv2d_16 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 14, 14, 64)        0         
 ng2D)                                                           
                                    

2023-12-24 15:36:02.833624: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_5/dropout_20/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


248/248 [==============================] - 36s 136ms/step - loss: 1.2626 - accuracy: 0.4472 - val_loss: 1.2485 - val_accuracy: 0.4566
Epoch 2/10
248/248 [==============================] - 29s 116ms/step - loss: 1.0661 - accuracy: 0.5645 - val_loss: 1.2102 - val_accuracy: 0.4818
Epoch 3/10
248/248 [==============================] - 29s 116ms/step - loss: 0.9791 - accuracy: 0.6018 - val_loss: 1.1956 - val_accuracy: 0.4919
Epoch 4/10
248/248 [==============================] - 29s 116ms/step - loss: 0.9353 - accuracy: 0.6246 - val_loss: 1.0745 - val_accuracy: 0.5439
Epoch 5/10
248/248 [==============================] - 29s 118ms/step - loss: 0.9010 - accuracy: 0.6420 - val_loss: 1.0480 - val_accuracy: 0.5712
Epoch 6/10
248/248 [==============================] - 29s 116ms/step - loss: 0.8787 - accuracy: 0.6503 - val_loss: 1.0325 - val_accuracy: 0.5798
Epoch 7/10
248/248 [==============================] - 29s 115ms/step - loss: 0.8605 - accuracy: 0.6640 - val_loss: 1.0224 - val_accuracy: 0.5

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [30]:
model = keras.models.load_model('classification_net.h5')

unlabeled_data_dir = '/kaggle/input/shitpy/dataset/test'
# putting images in a list
unlabeled_images = os.listdir(unlabeled_data_dir)

unlabeled_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
unlabeled_generator = unlabeled_datagen.flow_from_dataframe(
    pd.DataFrame({'Image': unlabeled_images}),
    directory=unlabeled_data_dir,
    x_col='Image',
    y_col=None,
    class_mode=None,
    target_size=(64, 64),
    batch_size=32,
    shuffle=False 
)

# Predictions
predictions = model.predict(unlabeled_generator)

# Assign predictions to it's classes
predicted_labels = np.argmax(predictions, axis=1)

predictions_df = pd.DataFrame({
    'Image': unlabeled_generator.filenames,
    'Predicted_Label': predicted_labels
})

csv_save_path = 'unlabeled_predictions.csv'
predictions_df.to_csv("CRY.csv", index=False)

Found 100 validated image filenames.
4/4 [==============================] - 0s 47ms/step


In [31]:
df = pd.read_csv('/kaggle/working/CRY.csv')
v = df.rename(columns = {'Image':'image_id','Predicted_Label':'label'})

In [32]:
v.to_csv('CRY.csv',index=False)